<a href="https://colab.research.google.com/github/lamtung1997/Solving_PDEs_Deep_learning/blob/main/1_Poisson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# SETUP
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import time
import math
import itertools
pi = math.pi

In [2]:
# MODEL
inputs = keras.Input(shape=(2,), name='points')
hidden_1 = layers.Dense(64, activation='sigmoid', name='hidden_1')(inputs)
hidden_2 = layers.Dense(64, activation='sigmoid', name='hidden_2')(hidden_1)
hidden_3 = layers.Dense(64, activation='sigmoid', name='hidden_3')(hidden_2)
outputs = layers.Dense(1, activation=tf.math.square, name="fx")(hidden_3)
model = keras.Model(inputs=inputs, outputs=outputs, name='poisson')

In [3]:
# DATA
import itertools

n = 30                      # number of points in a border
c = np.linspace(0, 1, n)    # coordinates from 0 to 1 with uniform distribution
P_in, P_border = [],[]
for i, j in itertools.product(range(n), range(n)):
    p = [c[i], c[j]]
    if(i == 0 or i == n-1 or j == 0 or j == n-1):
        P_border.append(p)
    else:
        P_in.append(p)

# upsampling P_border that its size is equal to P_in
for i in range(len(P_in)-len(P_border)):
    j = i%(len(P_in))
    P_border.append(P_border[j])

# set batch_size and shuffle_size
batch_size = int(len(P_border)/392)
bs = int(len(P_border))

# convert numpy array, P_in and P_border to Tensorflow Dataset
P_in = tf.convert_to_tensor(P_in, dtype=tf.float32)
P_border = tf.convert_to_tensor(P_border, dtype=tf.float32)

P_in = tf.data.Dataset.from_tensor_slices((P_in))
P_in = P_in.shuffle(buffer_size=bs).batch(batch_size)

P_border = tf.data.Dataset.from_tensor_slices((P_border))
P_border = P_border.shuffle(buffer_size=bs).batch(batch_size)

In [4]:
# CHOOSE OPTIMIZER
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

In [5]:
# TRAIN FUNCTION
@tf.function
def train_step(P_in, P_border):
    with tf.GradientTape(persistent=True) as tape:
        loss_in, loss_border = [0], [0]
        # interior loss
        for i in range(P_in.shape[0]):
            v = tf.reshape(P_in[i], shape=(1,2))
            tape.watch(v)
            model_vals = model(v)

            # calculating u_xx and u_yy
            u_x_y = tape.gradient(model_vals, v)
            u_xx = tape.gradient(u_x_y[0][0], v)[0][0]
            u_yy = tape.gradient(u_x_y[0][1], v)[0][1]

            # calculating f(x,y)
            f = -2*pi*pi* tf.math.sin(pi*v[0][0]) * tf.math.sin(pi*v[0][1])

            # calculating loss_in
            loss_in += tf.math.square(u_xx + u_yy - f)

        # boundary loss
        for j in range(P_border.shape[0]):
            v = tf.reshape(P_border[j], shape=(1,2))
            loss_border += tf.math.square(model(v))
        
        # loss function
        batch_loss = loss_in + loss_border
    
    # update weights
    grads = tape.gradient(batch_loss, model.trainable_weights)          # gradient of loss function with respect to w
    optimizer.apply_gradients(zip(grads, model.trainable_weights))      # update w

    return batch_loss

In [6]:
# TRAIN
epochs = 2000
losses = []
start_time = time.time()
for epoch in range(epochs+1):

    optimizer.learning_rate.assign(1/(10**(int(epoch/400) + 2)))

    loss = 0
    for (P_in_batch,P_border_batch) in itertools.zip_longest(P_in,P_border):
        batch_loss = train_step(P_in_batch, P_border_batch)
        loss += batch_loss
    losses.append(loss.numpy())
    if(epoch%100 == 0):
        print("[%4s] Training loss: %10f \t learning_rate: %f" % (epoch, float(loss), optimizer.learning_rate.numpy()))

print("Total time: %.2f minutes" % ((time.time() - start_time)/60))

[  0] Training loss: 40404.644531 	 learning_rate: 0.010000
[100] Training loss: 350.609192 	 learning_rate: 0.010000
[200] Training loss: 243.876877 	 learning_rate: 0.010000
[300] Training loss: 163.018570 	 learning_rate: 0.010000
[400] Training loss:  46.793743 	 learning_rate: 0.001000
[500] Training loss:  22.267120 	 learning_rate: 0.001000
[600] Training loss:  17.741863 	 learning_rate: 0.001000
[700] Training loss:  14.452739 	 learning_rate: 0.001000
[800] Training loss:   8.619843 	 learning_rate: 0.000100
[900] Training loss:   7.950186 	 learning_rate: 0.000100
[1000] Training loss:   7.699492 	 learning_rate: 0.000100
[1100] Training loss:   7.479762 	 learning_rate: 0.000100
[1200] Training loss:   7.052985 	 learning_rate: 0.000010
[1300] Training loss:   6.942818 	 learning_rate: 0.000010
[1400] Training loss:   6.925884 	 learning_rate: 0.000010
[1500] Training loss:   6.909665 	 learning_rate: 0.000010
[1600] Training loss:   6.845490 	 learning_rate: 0.000001
[1700

In [12]:
z = tf.convert_to_tensor([[0.0, 0.0]])
print("f(0.0, 0.0) \t = %.2f" % model(z).numpy()[0][0])
z = tf.convert_to_tensor([[0.1, 0.1]])
print("f(0.1, 0.1) \t = %.2f" % model(z).numpy()[0][0])
z = tf.convert_to_tensor([[0.2, 0.2]])
print("f(0.2, 0.2) \t = %.2f" % model(z).numpy()[0][0])
z = tf.convert_to_tensor([[0.3, 0.3]])
print("f(0.3, 0.3) \t = %.2f" % model(z).numpy()[0][0])
z = tf.convert_to_tensor([[0.4, 0.4]])
print("f(0.4, 0.4) \t = %.2f" % model(z).numpy()[0][0])
z = tf.convert_to_tensor([[0.5, 0.5]])
print("f(0.5, 0.5) \t = %.2f" % model(z).numpy()[0][0])

f(0.0, 0.0) 	 = 0.04
f(0.1, 0.1) 	 = 0.15
f(0.2, 0.2) 	 = 0.41
f(0.3, 0.3) 	 = 0.73
f(0.4, 0.4) 	 = 0.99
f(0.5, 0.5) 	 = 1.08


In [ ]:
# model.save("/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/1. Poisson")

In [ ]:
model1 = keras.models.load_model("/content/drive/My Drive/Colab Notebooks/Keras/Solving PDEs/1. Poisson")